In [93]:
import datetime as dt
from lxml import html
from bs4 import BeautifulSoup
import urllib
import sys
import requests
import operator
from datetime import timedelta
import csv
import pandas as pd
import numpy as np
import math

In [94]:
year1617 = pd.read_csv('getting_SOS_games/season1617.csv', parse_dates=['Date'])
year1718 = pd.read_csv('getting_SOS_games/season1718.csv', parse_dates=['Date'])
year1819 = pd.read_csv('getting_SOS_games/season1819.csv', parse_dates=['Date'])
year1920 = pd.read_csv('getting_SOS_games/season1920.csv', parse_dates=['Date'])

In [95]:
frames = [year1617,year1718,year1819,year1920]

In [96]:
allyears = pd.concat(frames)
allyears = allyears.drop_duplicates(subset=['Date', 'Home'],keep='last')


In [97]:
allyears

,Unnamed: 0,Date,Win_Score,Lose_Score,Home,Away,WinnerHome,KenPomRank_Away,KenPomVal_Away,BPIrank_Away,...,d_eff_Home,ppg_Home,papg_Home,Pace_Home,Three_Par_Home,True_Shoot_Home,Tov_Home,Opp_Three_Par_Home,Opp_True_Shoot_Home,Opp_Tov_Home
0,0,2016-11-11,94,49,Duke,Marist,1,312.0,-13.63,305.0,...,91.17,80.8,70.2,69.9,0.383,0.588,14.2,0.288,0.510,14.9
1,1,2016-11-11,87,64,Kentucky,Stephen F. Austin,1,233.0,-6.19,262.0,...,86.87,84.9,71.5,74.3,0.316,0.564,13.4,0.333,0.507,16.9
2,2,2016-11-11,103,99,Kansas,Indiana,0,44.0,14.49,32.0,...,89.65,83.2,71.9,71.4,0.359,0.583,15.1,0.342,0.515,16.1
3,3,2016-11-11,88,48,Villanova,Lafayette,1,330.0,-16.79,330.0,...,87.45,77.2,62.7,65.6,0.433,0.612,15.0,0.400,0.506,17.6
4,4,2016-11-11,91,77,Oregon,Army,1,230.0,-6.04,253.0,...,88.20,78.9,65.8,68.8,0.388,0.582,14.7,0.378,0.494,16.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298,5298,2020-03-11,74,69,Alabama-Birmingham,Texas-San Antonio,1,210.0,-3.84,230.0,...,99.65,13.0,67.3,67.2,0.300,0.537,19.0,0.409,0.514,16.2
5299,5299,2020-03-11,71,69,Oregon State,Utah,1,114.0,5.56,117.0,...,111.51,13.0,71.2,66.6,0.349,0.559,15.3,0.364,0.529,16.2
5300,5300,2020-03-11,86,73,Arkansas,Vanderbilt,1,169.0,0.14,165.0,...,109.40,12.0,75.8,71.4,0.399,0.553,14.4,0.333,0.516,19.2
5301,5301,2020-03-11,77,70,Arizona,Washington,1,54.0,12.67,70.0,...,113.66,11.0,76.4,70.3,0.347,0.546,14.5,0.377,0.498,18.1


In [98]:
full = allyears.dropna()

In [99]:
full

,Unnamed: 0,Date,Win_Score,Lose_Score,Home,Away,WinnerHome,KenPomRank_Away,KenPomVal_Away,BPIrank_Away,...,d_eff_Home,ppg_Home,papg_Home,Pace_Home,Three_Par_Home,True_Shoot_Home,Tov_Home,Opp_Three_Par_Home,Opp_True_Shoot_Home,Opp_Tov_Home
0,0,2016-11-11,94,49,Duke,Marist,1,312.0,-13.63,305.0,...,91.17,80.8,70.2,69.9,0.383,0.588,14.2,0.288,0.510,14.9
1,1,2016-11-11,87,64,Kentucky,Stephen F. Austin,1,233.0,-6.19,262.0,...,86.87,84.9,71.5,74.3,0.316,0.564,13.4,0.333,0.507,16.9
2,2,2016-11-11,103,99,Kansas,Indiana,0,44.0,14.49,32.0,...,89.65,83.2,71.9,71.4,0.359,0.583,15.1,0.342,0.515,16.1
3,3,2016-11-11,88,48,Villanova,Lafayette,1,330.0,-16.79,330.0,...,87.45,77.2,62.7,65.6,0.433,0.612,15.0,0.400,0.506,17.6
4,4,2016-11-11,91,77,Oregon,Army,1,230.0,-6.04,253.0,...,88.20,78.9,65.8,68.8,0.388,0.582,14.7,0.378,0.494,16.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298,5298,2020-03-11,74,69,Alabama-Birmingham,Texas-San Antonio,1,210.0,-3.84,230.0,...,99.65,13.0,67.3,67.2,0.300,0.537,19.0,0.409,0.514,16.2
5299,5299,2020-03-11,71,69,Oregon State,Utah,1,114.0,5.56,117.0,...,111.51,13.0,71.2,66.6,0.349,0.559,15.3,0.364,0.529,16.2
5300,5300,2020-03-11,86,73,Arkansas,Vanderbilt,1,169.0,0.14,165.0,...,109.40,12.0,75.8,71.4,0.399,0.553,14.4,0.333,0.516,19.2
5301,5301,2020-03-11,77,70,Arizona,Washington,1,54.0,12.67,70.0,...,113.66,11.0,76.4,70.3,0.347,0.546,14.5,0.377,0.498,18.1


In [100]:
full['Home_Score_Dif'] = np.where(full['WinnerHome']==1,full['Win_Score']-full['Lose_Score'],full['Lose_Score']-full['Win_Score'])

c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
y = 0
for x in full.columns:
    print(x,y)
    y+=1

Unnamed: 0 0
Date 1
Win_Score 2
Lose_Score 3
Home 4
Away 5
WinnerHome 6
KenPomRank_Away 7
KenPomVal_Away 8
BPIrank_Away 9
BPIval_Away 10
SRS_Away 11
SOS_Away 12
o_eff_Away 13
d_eff_Away 14
ppg_Away 15
papg_Away 16
Pace_Away 17
Three_Par_Away 18
True_Shoot_Away 19
Tov_Away 20
Opp_Three_Par_Away 21
Opp_True_Shoot_Away 22
Opp_Tov_Away 23
KenPomRank_Home 24
KenPomVal_Home 25
BPIrank_Home 26
BPIval_Home 27
SRS_Home 28
SOS_Home 29
o_eff_Home 30
d_eff_Home 31
ppg_Home 32
papg_Home 33
Pace_Home 34
Three_Par_Home 35
True_Shoot_Home 36
Tov_Home 37
Opp_Three_Par_Home 38
Opp_True_Shoot_Home 39
Opp_Tov_Home 40
Home_Score_Dif 41


In [102]:
X = full.iloc[:,[12,13,14,17,18,19,20,21,22,23,29,30,31,34,35,36,37,38,39,40]]
# X = full.iloc[:,[11,12,15,16,17,18,19,20,21,27,28,31,32,33,34,35,36,37]]
# X = full.iloc[:,[7,9,10,23,25,26]]

y = full.iloc[:,6]

In [103]:
X

,SOS_Away,o_eff_Away,d_eff_Away,Pace_Away,Three_Par_Away,True_Shoot_Away,Tov_Away,Opp_Three_Par_Away,Opp_True_Shoot_Away,Opp_Tov_Away,SOS_Home,o_eff_Home,d_eff_Home,Pace_Home,Three_Par_Home,True_Shoot_Home,Tov_Home,Opp_Three_Par_Home,Opp_True_Shoot_Home,Opp_Tov_Home
0,-2.74,96.94,112.97,69.1,0.421,0.539,16.6,0.347,0.564,13.5,10.91,122.70,91.17,69.9,0.383,0.588,14.2,0.288,0.510,14.9
1,-5.75,94.04,102.27,69.3,0.345,0.544,21.6,0.346,0.547,20.5,10.49,120.28,86.87,74.3,0.316,0.564,13.4,0.333,0.507,16.9
2,8.16,117.34,96.09,69.5,0.387,0.582,18.2,0.342,0.525,13.8,12.23,123.67,89.65,71.4,0.359,0.583,15.1,0.342,0.515,16.1
3,-3.50,93.35,112.40,70.3,0.400,0.528,18.8,0.437,0.563,16.1,9.28,123.50,87.45,65.6,0.433,0.612,15.0,0.400,0.506,17.6
4,-4.10,96.97,103.30,72.0,0.433,0.538,18.3,0.349,0.529,15.4,8.38,118.40,88.20,68.8,0.388,0.582,14.7,0.378,0.494,16.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298,0.41,-3.18,102.75,74.0,0.424,0.526,15.0,0.392,0.540,16.4,-1.64,-0.94,99.65,67.2,0.300,0.537,19.0,0.409,0.514,16.2
5299,7.86,8.47,107.62,67.7,0.383,0.550,17.3,0.353,0.529,15.5,6.15,9.70,111.51,66.6,0.349,0.559,15.3,0.364,0.529,16.2
5300,6.41,3.16,105.80,70.0,0.443,0.540,16.4,0.338,0.564,17.2,7.27,13.46,109.40,71.4,0.399,0.553,14.4,0.333,0.516,19.2
5301,8.25,12.41,107.14,70.2,0.384,0.543,18.8,0.382,0.482,16.6,8.40,19.49,113.66,70.3,0.347,0.546,14.5,0.377,0.498,18.1


In [104]:
import numpy as np
# Labels are the values we want to predict
labels = np.array(y)
# Saving feature names for later use
feature_list = list(X.columns)
# Convert to numpy array
features = np.array(X)

In [105]:
feature_list

['SOS_Away',
 'o_eff_Away',
 'd_eff_Away',
 'Pace_Away',
 'Three_Par_Away',
 'True_Shoot_Away',
 'Tov_Away',
 'Opp_Three_Par_Away',
 'Opp_True_Shoot_Away',
 'Opp_Tov_Away',
 'SOS_Home',
 'o_eff_Home',
 'd_eff_Home',
 'Pace_Home',
 'Three_Par_Home',
 'True_Shoot_Home',
 'Tov_Home',
 'Opp_Three_Par_Home',
 'Opp_True_Shoot_Home',
 'Opp_Tov_Home']

In [106]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

In [107]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 42)

print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (15750, 20)
Training Labels Shape: (15750,)
Testing Features Shape: (5250, 20)
Testing Labels Shape: (5250,)


In [108]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 5250 points : 1400


In [109]:
gnb.score(X_test, y_test)

0.7333333333333333

In [110]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 977,  894],
       [ 506, 2873]], dtype=int64)

In [111]:
# from sklearn.inspection import permutation_importance

# imps = permutation_importance(gnb, X_test, y_test)
# print(imps.importances_mean)
# feature_imp = pd.Series(imps.importances_mean,index=X.columns).sort_values(ascending=False)
# feature_imp

In [112]:
# ISUatKU = [97.23,96,79.9,.43,.516,16.7,.537,.446,20.8,80.84,110.41,75.5,.22,.576,15.1,.374,.447,17.5]
ISUatKU = [18.4,6.9,14.42,16.52,6.6,12.85]
game = [ISUatKU]

In [113]:
y_pred_gnb=gnb.predict_proba(game)

ValueError: operands could not be broadcast together with shapes (1,6) (20,) 

In [ ]:
y_pred_gnb

In [ ]:
(5043-1206)/5043